In [1]:
import re
from collections import defaultdict
import json
import os
# import matplotlib.pyplot as plt
# import numpy as np
from outils import read, keys, load_cn_json, dump_cn_json, 中转数, 数转中, 中文标点, headtail, nice_print, sort_dict_with, dump_cn_json_compact, numsec, sectime

In [2]:
def behead(s, dilim):
    parts = s.split(dilim)
    return dilim.join(parts[1:])

def parse_cl_classical_chinese(text):
    # 初始化数据结构
    data = {
        "metadata": {
            "title": "",
            "author": "",
            "period": "",
            "provenance": "",
            "remark": "",
            "version": "1.0",
            "last_modified": numsec()
        },
        "content": {
            "raw": "",
            "structured": [],
            "text": []
        },
        "annotations": {
            "tokens": {},
            "sentences": [],
            "footnotes": []
        },
        "teaching_materials": {
            "vocabulary": [],
            "sentences": []
        }
    }

    # 解析元数据
    sections = re.split(r'\n(?=标题：|作者：|出处：|朝代：|介绍：|正文：|内文注释：|译文：|教学重点：)', text)
    section_map = {}
    current_section = None
    for section in sections:
        if section.startswith('标题：'):
            current_section = 'title'
            section_map[current_section] = behead(section, "：").strip()
        elif section.startswith('作者：'):
            section_map['author'] = behead(section, "：").strip()
        elif section.startswith('出处：'):
            section_map['source'] = behead(section, "：").strip()
        elif section.startswith('朝代：'):
            section_map['period'] = behead(section, "：").strip()
        elif section.startswith('介绍：'):
            current_section = 'intro'
            section_map[current_section] = behead(section, "：").strip()
        elif section.startswith('正文：'):
            current_section = 'content'
            section_map[current_section] = behead(section, "：").strip()
        elif section.startswith('内文注释：'):
            current_section = 'footnotes'
            section_map[current_section] = behead(section, "：").strip()
        elif section.startswith('译文：'):
            current_section = 'translation'
            section_map[current_section] = behead(section, "：").strip()
        elif section.startswith('教学重点：'):
            current_section = 'teachings'
            section_map[current_section] = behead(section, "：").strip()

    # 填充元数据
    data["metadata"]["title"] = section_map.get('title', '')
    data["metadata"]["author"] = section_map.get('author', '')
    data["metadata"]["provenance"] = section_map.get('source', '')
    data["metadata"]["period"] = section_map.get('period', '')
    data["metadata"]["remark"] = '\n\n'.join([p.strip() for p in section_map.get('intro', '').split('\n\n')])

    # 处理注释
    footnotes = {}
    if 'footnotes' in section_map:
        for line in section_map['footnotes'].split('\n'):
            if match := re.match(r'^N(\d+)：〔(.+?)〕(.+)', line.strip()):
                num, term, note = match.groups()
                footnotes[f'N{num}'] = {
                    'term': term,
                    'content': note.strip()
                }

    # 处理正文
    # 新断句逻辑：匹配以。！？结尾，且可能跟随右引号的模式
    def split_sentences(text):
        sentence_end_re = re.compile(r'([。！？](?:”)?)')
        splits = sentence_end_re.split(text)
        sentences = []
        buffer = []
        for seg in splits:
            buffer.append(seg)
            if sentence_end_re.fullmatch(seg):
                sentences.append(''.join(buffer))
                buffer = []
        if ''.join(buffer):
            sentences.append(''.join(buffer))
        return sentences

    # 段落处理函数
    def process_paragraph(para, token_counter=1):
        footnote_markers = list(re.finditer(r'<footnote:(N\d+)>', para))
        
        # 计算有效字符位置
        # offset_map = []
        current_offset = 0
        cleaned_chars = []
        for m in footnote_markers:
            start, end = m.span()
            cleaned_chars.extend(para[current_offset:start])
            current_offset = end
        cleaned_chars.extend(para[current_offset:])
        cleaned_para = ''.join(cleaned_chars)

        # 构建段落结构
        para_id = f"P{len(data['content']['structured']) + 1}"
        para_structure = {
            "para_id": para_id,
            "sentences": []
        }
        anno_para = {
            "para_id": para_id,
            "translations": []
        }

        # 分割句子
        sentences = split_sentences(cleaned_para)

        # 生成字符映射
        for sent_idx, sent in enumerate(sentences):
            sentence_id = f"{para_id.replace(':', '')}-S{sent_idx+1}"
            sentence_structure = {
                "sentence_id": sentence_id,
                "text": sent,
                # "char_pos": []
            }
            anno_sent = {
                "sentence_id": sentence_id,
                "text": sent,
                "translation": "",
            }

            for char in sent:
                if char not in 中文标点:
                    token_id = sentence_id + f"-T{token_counter}"
                    data["annotations"]["tokens"][token_id] = {
                        "char": char,
                        "pinyin": "",
                        "explanation": ""
                    }
                token_counter += 1

            para_structure["sentences"].append(sentence_structure)
            anno_para["translations"].append(anno_sent)

        data["content"]["structured"].append(para_structure)
        data['content']['text'].append(cleaned_para)
        data["annotations"]['sentences'].append(anno_para)
        
        # 计算脚注位置
        footnote_positions = []
        for m in footnote_markers:
            note_id = m.group(1)
            original_start = m.start()
            # 计算有效位置
            valid_pos = len(re.sub(r'<footnote:N\d+>', '', para[:original_start]))
            footnote_positions.append((note_id, para_id, valid_pos))
        
        return footnote_positions
    
    # 处理正文内容
    all_footnotes = []
    raw_paragraphs = [p.strip() for p in section_map.get("content", "").split('\n\n')]
    data["content"]["raw"] = '\n\n'.join(raw_paragraphs)
    
    for para in raw_paragraphs:
        para_footnotes = process_paragraph(para)
        all_footnotes.extend(para_footnotes)

    # 生成内文注释
    for note_id, para_id, pos in all_footnotes:
        if note_info := footnotes.get(note_id):
            data["annotations"]["footnotes"].append({
                "note_id": note_id,
                "para_id": para_id,
                "footnote_pos": pos,
                "content": f"〔{note_info['term']}〕{note_info['content']}"
            })
    
    # 处理译文
    if 'translation' in section_map:
        for para, trans_para in zip(data["annotations"]["sentences"], section_map['translation'].split('\n\n')):
            # 分割句子
            for anno, sent in zip(para['translations'], split_sentences(trans_para)):
                anno['translation'] = sent

    # 处理教学重点内容
    if 'teachings' in section_map:
        for p in section_map['teachings'].split('\n\n'):
            if match := re.match(r'^V(\d+)：〔(.+?)〕(.+)', p.strip()):
                    num, term, note = match.groups()
                    teaching_note = {
                        'teaching_id': f'V{num}',
                        'content': term,
                        'teaching_note': note.strip()
                    }
                    data["teaching_materials"]["sentences"].append(teaching_note)

    return data

def refine(data):
    new_structured = {}
    for p in data["content"]["structured"]:
        for s in p["sentences"]:
            new_structured[s["sentence_id"]] = s["text"]
    data["content"]["sentences"] = new_structured
    del data["content"]["structured"]

    new_tranlsations = {}
    for p in data["annotations"]["sentences"]:
        for s in p["translations"]:
            new_tranlsations[s["sentence_id"]] = {"text": s["text"], "translation": s["translation"]}
    del data["annotations"]["sentences"]
    data["annotations"]["sentences"] = new_tranlsations

    new_footnotes = {}
    for fn in data["annotations"]["footnotes"]:
        new_footnotes[fn["note_id"]] = {}
        for k in fn:
            if k != "note_id":
                new_footnotes[fn["note_id"]][k] = fn[k]
    del data["annotations"]["footnotes"]
    data["annotations"]["footnotes"] = new_footnotes

    new_materials = {}
    for tm in data["teaching_materials"]["sentences"]:
        new_materials[tm["teaching_id"]] = {"text": tm["content"], "note": tm["teaching_note"]}
    del data["teaching_materials"]["sentences"]
    data["teaching_materials"]["sentences"] = new_materials

    del data["annotations"]['tokens']

    return data

def last_modified(data):
    """显示最新更改时间"""
    last_modified = data["last_modified"]
    print(f'最后修改于{sectime(last_modified)}')
    return last_modified

def update(data):
    """更新时间"""
    data["last_modified"] = numsec()

def upgrade_version(version="", part=-1):
    """提升版本号的指定段。

    参数:
        version (str): 原始版本号，格式为 "major.minor.patch"。
        part (int): 要提升的段，0 表示 major，1 表示 minor，2 表示 patch。

    返回:
        str: 新的版本号。
    """
    # 将版本号字符串分割成列表
    if not version:
        parts = [0,0,0]
    elif "." not in version:
        parts = [int(version),0,0]
    else:
        parts = list(map(int, version.split('.')))
        if len(parts) < 3:
            parts.append(0)

    if part == -1:
        part = 2
    
    # 检查 part 是否在有效范围内
    if part < 0 or part > 2:
        raise ValueError("part 参数超出版本号段的范围")
    
    # 提升指定段的值，并将后续段重置为 0
    parts[part] += 1
    for i in range(part + 1, len(parts)):
        parts[i] = 0
    
    # 将列表转换回字符串格式
    return '.'.join(map(str, parts))

def text_str(data, title=True, author=0, sep="\n\n"):
    """打印课文（不含注释）
    """
    out = ""
    if title:
        out += data["metadata"]["title"]
        out += sep

    if author:
        author_str = ""
        if author > 1 and data["metadata"]["period"]:
            author_str += "〔" + data["metadata"]["period"] + "〕"
        author_str += data["metadata"]["author"]
        out += author_str
        out += sep
        
    out += sep.join(data["content"]["text"])
    return out

def text_tex_old(data, title=True, author=0, sep="\n\n"):
    """打印课文（tex格式）
    """
    out = ""
    if title:
        out += data["metadata"]["title"]
        out += sep

    if author:
        author_str = ""
        if author > 1 and data["metadata"]["period"]:
            author_str += "〔" + data["metadata"]["period"] + "〕"
        author_str += data["metadata"]["author"]
        out += author_str
        out += sep

    paragraphs_ready = data["content"]["raw"].split(sep)
    for footnote in data['annotations']['footnotes']:
        n_para = int(footnote["para_id"][1:]) - 1
        p = paragraphs_ready[n_para]
        paragraphs_ready[n_para] = p.replace("<footnote:"+footnote['note_id']+">", r"\footnote{" + footnote['content'] + "}")

    out += sep.join(paragraphs_ready)
    return out

def text_tex(data, title=True, author=0, sep="\n\n"):
    """打印课文（tex格式）
    """
    out = ""
    if title:
        out += data["metadata"]["title"]
        out += sep

    if author:
        author_str = ""
        if author > 1 and data["metadata"]["period"]:
            author_str += "〔" + data["metadata"]["period"] + "〕"
        author_str += data["metadata"]["author"]
        out += author_str
        out += sep

    paragraphs_ready = data["content"]["raw"].split(sep)
    for nid, footnote in data['annotations']['footnotes'].items():
        n_para = int(footnote["para_id"][1:]) - 1
        p = paragraphs_ready[n_para]
        paragraphs_ready[n_para] = p.replace("<footnote:"+nid+">", r"\footnote{" + footnote['content'] + "}")

    out += sep.join(paragraphs_ready)
    return out

def read_text(path, refined=True):
    data = parse_cl_classical_chinese("".join([line for line in read(path) if not line.startswith("#")]))
    if refined:
        data = refine(data)
    return data

# 示例使用
sample_text = """
标题：小石潭记

作者：柳宗元

朝代：唐

出处：《柳河东集》第29卷（中华书局1974年版）

介绍：柳宗元于唐顺宗永贞元年（公元805年）因拥护王叔文的改革，被皇帝贬为永州司马，王叔文被害。政治上的失意，使他寄情于山水，并通过对景物的具体描写，抒发自己的被贬后无法排遣的忧伤凄苦的思想感情，成为后世写作山水游记的楷模。 此间共写了8篇知名的山水游记：《始得西山宴游记》《钴鉧潭记》《钴鉧潭西小丘记》《小石潭记》《袁家渴记》《石渠记》《石涧记》《小石城山记》，后称《永州八记》。在第一篇《始得西山宴游记》中作者这样记述当时的心情：“自余为僇人，居是州，恒惴栗。” 《小石潭记》中景语即情语，“悄怆幽邃，凄神寒骨”处的描写，情景交融，很好地说明了这一问题 。《小石潭记》原题为《至小丘西小石潭记》，是《永州八记》之四。

正文：

从小丘<footnote:N1>西行百二十步，隔篁竹，闻水声，如鸣佩环，心乐之。伐竹取道，下见小潭，水尤清冽。全石以为底，近岸，卷石底以出，为坻，为屿，为嵁，为岩。青树翠蔓，蒙络摇缀，参差披拂。

潭中鱼可百许头，皆若空游无所依。日光下澈，“影布石上。”佁然不动，俶尔远逝，往来翕忽，似与游者相乐。

潭西南而望，斗折蛇行，明灭可见。其岸势犬牙差互，不可知其源。

坐潭上，四面竹树环合，寂寥无人，“凄神寒骨，悄怆幽邃？”以其境过清，不可久居，乃记之而去。

同游者：吴武陵<footnote:N2>，龚古<footnote:N3>，余弟宗玄<footnote:N4>。隶而从者，崔氏二小生：曰恕己，曰奉壹。

内文注释：

N1：〔小丘〕即钴鉧潭西小丘，见前一篇《钴鉧潭西小丘记》。

N2：〔吴武陵〕信州（今重庆奉节一带）人，唐宪宗元和初进士，因罪贬官永州，与作者友善。

N3：〔龚古〕作者朋友。

N4：〔宗玄〕作者的堂弟。

译文：

从小土丘往西走约一百二十步，隔着竹林，听到水声，好象挂在身上的玉佩、玉环相互碰撞的声音，心里很是高兴。（于是）砍伐竹子，开出一条道路，下面显现出一个小小的水潭，潭水特别清凉。潭以整块石头为底，靠近岸边，石底向上弯曲，露出水面，像各种各样的石头和小岛。青葱的树木，翠绿的藤蔓，遮掩缠绕，摇动下垂，参差不齐，随风飘动。　

潭中大约有一百来条鱼，都好像在空中游动，没有什么依靠似的。阳光往下一直照到潭底，鱼儿的影子映在水底的石上。（鱼儿）呆呆地静止不动，忽然间（又）向远处游去，来来往往，轻快敏捷，好像跟游人逗乐似的。

向石潭的西南方向望去，（溪流）像北斗七星那样的曲折，（又）像蛇爬行一样的蜿蜒，（有时）看得见，（有时）看不见。两岸的形状像犬牙似的参差不齐，看不出溪水的源头在哪里。

坐在石潭边上，四面被竹林树木包围着，静悄悄的，空无一人，（这气氛）使人感到心神凄凉，寒气透骨，幽静深远，弥漫着忧伤的气息。因为环境过于凄清，不能长时间地待下去，就记下这番景致离开了。

一同去游览的有吴武陵、龚古，我的弟弟宗玄。跟着一同去的还有姓崔的两个年轻人，一个叫恕己，一个叫奉壹。
"""

# result = refine(parse_cl_classical_chinese(sample_text))
filepath = r"D:\Documents\Projects\SchoolChinese\r\82\xiaoshitanji.md"
filepath = r"D:\Documents\Projects\SchoolChinese\r\81\dadaozhixing.md"
filepath = r"D:\Documents\Projects\SchoolChinese\r\81\guanchao.md"
filepath = r"D:\Documents\Projects\SchoolChinese\r\81\hezhouji.md"
filepath = r"D:\Documents\Projects\SchoolChinese\r\81\ailianshuo.md"
filepath = r"D:\Documents\Projects\SchoolChinese\s\3\hongmenyan.md"
filepath = r"D:\Documents\Projects\SchoolChinese\r\71\lunyu1.md"
result = read_text(filepath)
# print(text_tex(result, author=2))
result

{'metadata': {'title': '《论语》十则',
  'author': '孔丘等',
  'period': '战国',
  'provenance': '《论语》',
  'remark': '《论语》由孔子的弟子及其再传弟子编撰而成，是儒家学派的经典著作之一，集中体现了孔子的政治主张、伦理思想、道德观念和教育原则。这里摘取了孔子关于个人如何培养道德、学习知识的看法。',
  'version': '1.0',
  'last_modified': 803577749},
 'content': {'raw': '子曰：“学而时习之，不亦说乎？有朋自远方来，不亦乐乎？人不知而不愠，不亦君子<footnote:N1>乎？”\n\n子曰：“我非生而知之者，好古，敏以求之者也。”\n\n曾子<footnote:N2>曰：“吾日三省吾身：为人谋而不忠乎？与朋友交而不信乎？传不习乎？”\n\n子曰：“温故而知新，可以为师矣。”\n\n子曰：“学而不思则罔，思而不学则殆。”\n\n子曰：“由<footnote:N3>，诲汝知之乎！知之为知之，不知为不知，是知也。”\n\n子曰：“见贤思齐焉，见不贤而内自省也。”\n\n子曰：“三人行，必有我师焉。择其善者而从之，其不善者而改之。”\n\n子贡<footnote:N4>问曰：“有一言而可以终身行之者乎？”子曰：“其恕乎！己所不欲，勿施于人。”\n\n子在川上曰：“逝者如斯夫，不舍昼夜。”',
  'text': ['子曰：“学而时习之，不亦说乎？有朋自远方来，不亦乐乎？人不知而不愠，不亦君子乎？”',
   '子曰：“我非生而知之者，好古，敏以求之者也。”',
   '曾子曰：“吾日三省吾身：为人谋而不忠乎？与朋友交而不信乎？传不习乎？”',
   '子曰：“温故而知新，可以为师矣。”',
   '子曰：“学而不思则罔，思而不学则殆。”',
   '子曰：“由，诲汝知之乎！知之为知之，不知为不知，是知也。”',
   '子曰：“见贤思齐焉，见不贤而内自省也。”',
   '子曰：“三人行，必有我师焉。择其善者而从之，其不善者而改之。”',
   '子贡问曰：“有一言而可以终身行之者乎？”子曰：“其恕乎！己所不欲，勿施于人。”',
   '子在川上曰：“逝者如斯夫，不舍昼夜。”'],
  'sentences':

In [3]:
print(list(result.keys()))
# print(list(result['content'].keys()))
# print(list(result['content']['structured'][0].keys()))
# print(list(result['content']['structured'][0]['sentences'][0].keys()))
# result['content']['structured'][0]['sentences'][0]['char_pos']

print(list(result['annotations'].keys()))
print(list(result['annotations']['sentences'].keys()))
result['annotations']['sentences']

print(list(result['teaching_materials'].keys()))
if result['teaching_materials']['sentences']:
    print(list(result['teaching_materials']['sentences'].keys()))
    result['teaching_materials']['sentences']

['metadata', 'content', 'annotations', 'teaching_materials']
['sentences', 'footnotes']
['P1-S1', 'P1-S2', 'P1-S3', 'P2-S1', 'P2-S2', 'P2-S3', 'P2-S4', 'P2-S5', 'P2-S6', 'P2-S7', 'P2-S8', 'P2-S9', 'P2-S10', 'P2-S11', 'P2-S12', 'P2-S13', 'P3-S1', 'P3-S2', 'P3-S3', 'P3-S4', 'P3-S5', 'P3-S6', 'P3-S7', 'P3-S8', 'P3-S9', 'P3-S10', 'P4-S1', 'P4-S2', 'P4-S3', 'P4-S4', 'P4-S5', 'P4-S6', 'P4-S7', 'P4-S8', 'P4-S9', 'P4-S10', 'P4-S11', 'P4-S12', 'P5-S1', 'P5-S2', 'P5-S3', 'P5-S4', 'P5-S5', 'P5-S6', 'P6-S1', 'P6-S2', 'P6-S3', 'P6-S4', 'P6-S5', 'P6-S6', 'P6-S7', 'P6-S8', 'P6-S9', 'P7-S1', 'P7-S2', 'P8-S1', 'P8-S2', 'P9-S1', 'P9-S2', 'P9-S3', 'P9-S4', 'P9-S5', 'P9-S6', 'P9-S7', 'P10-S1', 'P10-S2', 'P10-S3', 'P10-S4', 'P10-S5', 'P10-S6', 'P10-S7', 'P10-S8', 'P10-S9', 'P10-S10', 'P10-S11', 'P10-S12', 'P10-S13', 'P10-S14', 'P10-S15', 'P10-S16', 'P11-S1', 'P12-S1', 'P12-S2', 'P12-S3', 'P12-S4', 'P12-S5', 'P12-S6', 'P12-S7', 'P12-S8', 'P13-S1', 'P13-S2', 'P13-S3', 'P13-S4', 'P13-S5', 'P13-S6', 'P13-S7', 

In [102]:
# dump_cn_json("example2.json", result)
dump_cn_json_compact("example.json", result, max_elements=50)
# result['annotations']['footnotes']

In [3]:
dirpath = "D:/Documents/Projects/SchoolChinese/"
# filepath = os.path.join(dirpath, str(), r"\ailianshuo.md")

data_cz = {}  # 初中古文课文
cats = {}
# 遍历目录及其子目录
for root, dirs, files in os.walk(dirpath):
    # 遍历文件列表
    for file in files:
        # 检查文件扩展名是否为 .md
        if file.endswith(".md"):
            # 获取文件的完整路径
            file_path = os.path.join(root, file)
            # print(os.path.splitext(file)[0])
            begs = [line[0] for line in read(file_path)]
            if not ">" in begs:
                # print(file_path)
                name = os.path.splitext(file)[0]
                if name not in data_cz:
                    data_cz[name] = read_text(file_path)
                    graw = root.split("/")[-1]
                    if graw.startswith("r"):
                        cats[name] = "chu"
                        gint = int(graw.split("\\")[-1])
                        data_cz[name]['teaching_materials']['grade'] = (gint % 10) + 2 * (gint // 10 - 7)
                    elif graw.startswith("s"):
                        cats[name] = "gao"
                        data_cz[name]['teaching_materials']['grade'] = int(graw.split("\\")[-1]) + 6
                    # data_cz[name] = 0
                else:
                    print(f"{name}.md 已经出现过，重复名字。")

dump_cn_json_compact("../src/中学/中学古文阅读课文.json", data_cz, max_elements=50)

In [76]:
pm = """烛之武退秦师 | 左丘明 | 必修一 | 第1课 | 人教版
荆轲刺秦王 | 刘向 | 必修一 | 第2课 | 人教版
兰亭集序 | 王羲之 | 必修二 | 第3课 | 人教版
赤壁赋 | 苏轼 | 必修二 | 第4课 | 人教版
劝学 | 荀子 | 必修三 | 第3课 | 人教版
过秦论 | 贾谊 | 必修三 | 第5课 | 人教版
陈情表 | 李密 | 必修五 | 第3课 | 人教版
《论语》十二则 | 孔子及其弟子 | 选择性必修上册 | 第1课 | 部编版
项脊轩志 | 归有光 | 选择性必修下册 | 第2课 | 部编版
六国论 | 苏洵 | 必修下册 | 第5课 | 部编版
阿房宫赋 | 杜牧 | 必修下册 | 第6课 | 部编版
石钟山记 | 苏轼 | 选择性必修下册 | 第4课 | 部编版
逍遥游 | 庄子 | 必修四 | 第3课 | 苏教版
鸿门宴 | 司马迁 | 必修一 | 第3课 | 人教版
滕王阁序 | 王勃 | 必修五 | 第2课 | 人教版
与妻书 | 林觉民 | 必修下册 | 第4课 | 部编版
老子四章 | 老子 | 选择性必修上册 | 第2课 | 部编版
兼爱 | 墨子 | 选择性必修上册 | 第3课 | 部编版
师说 | 韩愈 | 必修三 | 第4课 | 人教版
寡人之于国也 | 孟子 | 必修三 | 第2课 | 人教版
报任安书 | 司马迁 | 必修五 | 第1课 | 人教版
归去来兮辞 | 陶渊明 | 必修五 | 第4课 | 人教版
游褒禅山记 | 王安石 | 必修五 | 第5课 | 人教版
五人墓碑记 | 张溥 | 必修五 | 第6课 | 人教版
登泰山记 | 姚鼐 | 必修五 | 第7课 | 人教版
种树郭橐驼传 | 柳宗元 | 必修五 | 第8课 | 人教版
黄州快哉亭记 | 苏辙 | 必修五 | 第9课 | 人教版
前赤壁赋 | 苏轼 | 必修五 | 第10课 | 人教版
后赤壁赋 | 苏轼 | 必修五 | 第11课 | 人教版
送东阳马生序 | 宋濂 | 必修五 | 第12课 | 人教版
岳阳楼记 | 范仲淹 | 必修五 | 第13课 | 人教版
醉翁亭记 | 欧阳修 | 必修五 | 第14课 | 人教版
桃花源记 | 陶渊明 | 必修五 | 第15课 | 人教版
出师表 | 诸葛亮 | 必修五 | 第16课 | 人教版
勾践灭吴 | 国语 | 必修一 | 第2课 | 人教版
邹忌讽齐王纳谏 | 战国策 | 必修一 | 第3课 | 人教版
触龙说赵太后 | 战国策 | 必修一 | 第4课 | 人教版
子路曾皙冉有公西华侍坐 | 论语 | 必修一 | 第5课 | 人教版
季氏将伐颛臾 | 论语 | 必修一 | 第8课 | 人教版
谏太宗十思疏 | 魏徵 | 必修二 | 第6课 | 人教版
伶官传序 | 欧阳修 | 必修二 | 第10课 | 人教版
病梅馆记 | 龚自珍 | 必修二 | 第15课 | 人教版
谏逐客书 | 李斯 | 选择性必修 | 第4课 | 部编版
答司马谏议书 | 王安石 |  |  |
屈原列传 | 司马迁 |  |  |
始得西山宴游记 | 柳宗元 |  |  |
廉颇蔺相如列传 | 司马迁 |  |  |
苏武传 | 班固 |  |  |
张衡传 | 范晔 |  |  |
段太尉轶事状 | 柳宗元 |  |  |
郑伯克段于鄢 | 战国策 |  |  |
唐雎不辱使命 | 战国策 |  |  |
冯谖客孟尝君 | 战国策 |  |  |
邹忌讽齐王纳谏 |  |  |  |
孟子见梁惠王 | 战国策 |  |  |
曹刿论战 | 左传 |  |  |
子鱼论战 | 左传 |  |  |
召公谏厉王弭谤 | 《国语·周语上》 |  |  |
陋室铭 | 刘禹锡 |  |  |
隆中对 | 陈寿 | 《三国志》 |  |  |
乐毅报燕王书 | 《战国策》 |  |  |
论积贮疏 | 贾谊 |  |  |
论贵粟疏 | 晁错 |  |  |
原君 | 黄宗羲 《明夷待访录》 |  |  |"""

titles = set([line.split("|")[0].strip() for line in pm.split("\n")])
ttex1 = set([t["metadata"]["title"] for k, t in data_cz.items() if cats[k] == "chu"])
ttex2 = set([t["metadata"]["title"] for k, t in data_cz.items() if cats[k] == "gao"])

todo = titles - ttex1 - ttex2
print(len(todo), "TODO")
nice_print(list(todo))

print("_--------_")
print(len(ttex1) + len(ttex2), "=", len(ttex1), "+", len(ttex2))
print("_--------_")
nice_print(list(ttex1))
print("_--------_")
nice_print(list(ttex2))

18 TODO
['答司马谏议书', '勾践灭吴', '寡人之于国也', '报任安书', '赤壁赋']
['老子四章', '陈情表', '触龙说赵太后', '段太尉轶事状', '子路曾皙冉有公西华侍坐']
['荆轲刺秦王', '冯谖客孟尝君', '郑伯克段于鄢', '病梅馆记', '种树郭橐驼传']
['黄州快哉亭记', '送东阳马生序', '逍遥游']
_--------_
81 = 36 + 45
_--------_
['马说', '使琉球记', '童趣', '邹忌讽齐王纳谏', '鱼我所欲也']
['观潮', '记承天寺夜游', '醉翁亭记', '岳阳楼记', '与朱元思书']
['陈涉世家', '五柳先生传', '唐雎不辱使命', '大道之行也', '公输']
['《庄子》故事两则', '三峡', '满井游记', '出师表', '登泰山记']
['陋室铭', '《论语》十则', '桃花源记', '曹刿论战', '爱莲说']
['孙权劝学', '智子疑邻', '伤仲永', '湖心亭看雪', '《世说新语》两则']
['核舟记', '孟子见梁惠王', '口技', '天文地理', '小石潭记']
['隆中对']
_--------_
['始得西山宴游记', '师说', '屈原列传', '兰亭集序', '谏太宗十思疏']
['谏逐客书', '张衡传', '阿房宫赋', '过秦论', '鸿门宴']
['兼爱', '归去来兮辞', '黄花岗烈士事略序', '石钟山记', '原君']
['伶官传序', '牧民', '苏武传', '季氏将伐颛臾', '与妻书']
['秋水', '烛之武退秦师', '前赤壁赋', '五人墓碑记', '后赤壁赋']
['《论语》十二则', '孟子四则', '乐毅报燕王书', '论贵粟疏', '非攻']
['游褒禅山记', '利议', '滕王阁序', '《孟子》六则', '廉颇蔺相如列传']
['促织', '指南录后序', '子鱼论战', '五蠹', '项脊轩志']
['秋声赋', '六国论', '论积贮疏', '劝学', '召公谏厉王弭谤']


In [ ]:
zis = []
for k, data in data_cz.items():
    # print(type(data["content"]["text"]))
    text = "".join(data["content"]["text"])
    for zi in text:
        if zi not in 中文标点 and zi not in "1234567890〔〕.?!~:":
            zis.append(zi)

len(zis), len(set(zis))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

def analyze_chinese_chars(char_list, n=10):
    """
    分析汉字字符列表，统计频率并可视化
    
    参数:
        char_list: 包含汉字的列表
        n: 要显示的最高频率字符数量，默认为10
    
    返回:
        top_n_chars: 最高频的n个汉字及其频率
    """
    # 统计每个汉字出现的次数
    char_counter = Counter(char_list)
    
    # 获取总字符数
    total_chars = len(char_list)
    
    # 获取前n个最常见的汉字
    top_n = char_counter.most_common(n)
    
    # 准备结果数据
    top_n_chars = []
    for char, count in top_n:
        top_n_chars.append({
            'char': char,
            'count': count,
            'percentage': count / total_chars * 100
        })
    
    # 可视化
    chars = [item['char'] for item in top_n_chars]
    counts = [item['count'] for item in top_n_chars]
    percentages = [item['percentage'] for item in top_n_chars]
    
    # 设置中文字体
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
    
    # 根据n调整图表尺寸
    fig_height = max(5, n * 0.5)  # 随n增加而增加高度
    fig, ax = plt.subplots(figsize=(10, fig_height))
    
    # 创建索引用于位置
    y_pos = np.arange(len(top_n_chars))
    
    # 创建水平条形图，汉字在y轴，避免拥挤
    bars = ax.barh(y_pos, counts, color='skyblue', edgecolor='black')
    
    # 添加每条的百分比标签
    for i, (v, p) in enumerate(zip(counts, percentages)):
        ax.text(v + 5, y_pos[i] - 0.1, f"{p:.1f}%", fontsize=10)
    
    # 设置图表标题和标签
    ax.set_xlabel('出现次数')
    ax.set_ylabel('汉字')
    ax.set_title(f'前{len(top_n_chars)}个最频繁出现的汉字')
    ax.set_yticks(y_pos)
    ax.set_yticklabels(chars)
    
    # 调整布局，让标签显示完整
    plt.tight_layout()
    
    # 显示图表
    plt.show()
    
    return top_n_chars

# 示例用法
# 示例汉字列表{
chars = zis
n = 200
top_chars = analyze_chinese_chars(chars, n=n)

print("前10个最频繁出现的汉字:")
for item in top_chars:
    print(f"汉字: {item['char']}, 出现次数: {item['count']}, 占比: {item['percentage']:.1f}%")

In [118]:
t = 0.35
hotzis = []
for zi in top_chars:
    if zi['percentage'] > t:
        hotzis.append(zi["char"])
    
nice_print(hotzis)

['之', '不', '而', '以', '也']
['者', '人', '其', '于', '为']
['曰', '有', '王', '子', '所']
['下', '天', '无', '如', '君']
['大', '与', '一', '可', '则']
['此', '是', '日', '中', '故']
['矣', '相', '能', '臣', '吾']
['得', '秦', '自', '国', '使']
['公', '乎', '然']


In [119]:
虚词 = ['而', '何', '乎', '乃', '其', '且', '若', '所', '为', '焉', '也', '以', '因', '于', '与', '则', '者', '之']  # , '安', '但', '耳', '夫', '盖', '故', '或', '即', '既', '然', '虽', '遂', '惟', '已', '矣', '犹', '哉'

nice_print([zi for zi in hotzis if zi not in 虚词], n = 10)
print("_--------_")
nice_print([zi for zi in hotzis if zi in 虚词], n = 10)
print("_--------_")
nice_print([zi for zi in 虚词 if zi not in hotzis], n = 10)

['不', '人', '曰', '有', '王', '子', '下', '天', '无', '如']
['君', '大', '一', '可', '此', '是', '日', '中', '故', '矣']
['相', '能', '臣', '吾', '得', '秦', '自', '国', '使', '公']
['然']
_--------_
['之', '而', '以', '也', '者', '其', '于', '为', '所', '与']
['则', '乎']
_--------_
['何', '乃', '且', '若', '焉', '因']


In [128]:
for zi in 虚词:
    inhot = False
    for hzi in top_chars:
        if hzi["char"] == zi and hzi["percentage"] > 1:
            inhot = True
            print(zi, hzi["percentage"])
    if not inhot:
        print(zi, "------")
    

而 1.9083969465648856
何 ------
乎 ------
乃 ------
其 1.179291446467019
且 ------
若 ------
所 ------
为 1.0642982971227246
焉 ------
也 1.5487375220199648
以 1.6147974163241339
因 ------
于 1.1670581327069878
与 ------
则 ------
者 1.2477980035231944
之 3.9954002740262284


In [131]:
for ci in xuci:
    nlei = 0
    for lei in xuci[ci]:
        nlei += 1
        nsen = 0
        for yi in lei["yis"]:
            nsen += 1
            yi["label"] = f"{nlei}-{nsen}"

# xuci["之"]
dump_cn_json_compact("example.json", xuci["之"])

In [166]:
sids = {}
res = {}
for k, data in data_cz.items():
    sids[k] = {}
    res[k] = {}

In [190]:
zi = "之"

i = 0
j = 0
L = 1000
M = L * j
N = M + L
for k, data in data_cz.items():
    # print(type(data["content"]["text"]))
    for pid, sent in data["content"]["sentences"].items():
        if i == N:
            break
        if zi in sent:
            i += 1
            if i > M:
                print(f"{i}.", sent)
                sids[k][pid] = i

1. 子曰：“学而时习之，不亦说乎？
2. 子曰：“我非生而知之者，好古，敏以求之者也。”
3. 子曰：“由，诲汝知之乎！
4. 知之为知之，不知为不知，是知也。”
5. 择其善者而从之，其不善者而改之。”
6. 子贡问曰：“有一言而可以终身行之者乎？”
7. 漩皆圆，波浪密而细，如初筛之米，点点零落；“米糠”字，极有形容。
8. 问之琉球伙长，年已六十，往来海面八次，云此次最为简捷，而所见亦仅三山，即至姑米。
9. 取视之，长尺馀，绦甲朱髯、血睛火鬣，类世所画龙头，见之悚然！
10. 庖人乃以佳苏鱼进；问之，曰：“此非鱼名也，系削黑鳗鱼脊肉，干而为之”。
11. 食法，先以温水浸洗，裹蕉叶煨之，切片如刨花，连五、七片不断，又如兰花；宜清酱，颇有异味。
12. 惟求“佳苏”之义不得，适有长史至，问亦不解；因呼至前细核之，据云“此品在敝国既多且美，自王官以及贫民皆得食”；意殆谓如家常蔬菜，人人得食也。
13. 一名石筍崖，以形似名之也；石垣四周，垣后可望海，沿海多浮石，嵌空玲珑；潮水击之，声作钟磬。
14. 稍下为护国寺，国王祷雨之所也。
15. 长史复以花二盆见贻，标曰“水翁花”；视之，乃马兰花也。
16. 中山草木，多与中朝异称；盖因国中少书，多不识古来草木之名。
17. 如罗汉松，谓之㭴木；冬青，谓之福木；万寿菊，谓之禅菊：其初以意名之，后遂相沿不改。
18. 惜未携《群芳谱》来，一一证辨之耳！
19. 早起，偶步其侧，见新叶有蚀者；薄视之，有虫黄体而苍文，两角、八足，身方而毛，世所谓毛虫类。
20. 命仆捉而坑之，尽扫其巢。
21. 土人就形色呼之，无定名。
22. 闻初熟色青，以糠覆之则黄，与中国制柿无异。
23. 岁实为常，实如其须之数。
24. 卯刻，扬帆出那霸港；岸上、舟中送者如云，举手辞谢之。
25. 介山曰：“姑俟之！”
26. 然贼船小、我船大，彼络绎开帆，纵善驾驶，不能并集，犹一与一之势也。
27. 且既已遇之，惧亦无益！
28. 此我与汝致命之秋也，生死共之！”
29. 余举旗麾之，吴得进从舵门放子母炮，立毙四人，击喝者堕海；贼退不及，入百步，枪并发，又毙六人。
30. 一只乃退，二只又入三百步，复以炮击之，毙五人；稍进，又击之，复毙四人，乃退去。
31. 密令舵工将船稍横，俟大炮准对贼船，即施放一发，中之。
32. 遂解衣熟睡，付之不见不闻

In [189]:
with open("response.txt", encoding="utf-8") as f:
    lines = f.readlines()

for line in lines:
    num, reply = line.rstrip("\n").strip().split(".")
    for k in sids:
        for pid in sids[k]:
            if sids[k][pid] == int(num):
                res[k][pid] = [a.strip().strip('"') for a in reply.strip().strip("()").split(",") if a]

for k in res:
    for pid in res[k]:
        sent = data_cz[k]["content"]["sentences"][pid]
        if sent.count(zi) != len(res[k][pid]):
            print(sids[k][pid], k, pid, sent.count(zi), len(res[k][pid]), sent, res[k][pid])
# res["lunyu1"]

8 shiliuqiuji P6-S6 1 2 问之琉球伙长，年已六十，往来海面八次，云此次最为简捷，而所见亦仅三山，即至姑米。 ['1-1', '1-1']
17 shiliuqiuji P14-S5 4 2 如罗汉松，谓之㭴木；冬青，谓之福木；万寿菊，谓之禅菊：其初以意名之，后遂相沿不改。 ['2-1', '2-1']
41 tianwendili P5-S1 2 1 叁星为白虎之体，大火是苍龙之心。 ['2-1']
44 tianwendili P10-S1 2 1 宇宙之江山不改，古今之称谓各殊。 ['2-1']
49 tianwendili P19-S1 6 5 饶州之鄱阳，岳州之青草，润州之丹阳，鄂州之洞庭，苏州之太湖，此为天下之五湖。 ['2-1', '2-1', '2-1', '2-1', '2-1']
53 tongqu P2-S1 3 2 夏蚊成雷，私拟作群鹤舞于空中，心之所向，则或千或百，果然鹤也；昂首观之，项为之强。 ['1-1', '2-1']
72 shangzhongyong P3-S2 1 2 其受之天也，贤于材人远矣。 ['2-4', '1-1']
73 shangzhongyong P3-S3 1 2 卒之为众人，则其受于人者不至也。 ['2-4', '2-1']
77 ailianshuo P1-S4 1 2 予独爱莲之出淤泥而不然，濯清涟而不妖，中通外直，不蔓不枝，香远益清，亭亭净植，可远观而不可亵玩焉。 ['2-4', '2-4']
87 hezhouji P2-S5 2 1 闭之，则右刻“山高月小，水落石出”，左刻“清风徐来，水波不兴”，石青糁之。 ['1-1']


In [11]:
def split_chinese_sentences(text):
    sentences = []
    buffer = []
    quote_stack = []  # 用于跟踪嵌套引号状态
    i = 0
    n = len(text)
    
    while i < n:
        char = text[i]
        buffer.append(char)
        
        # 处理引号嵌套
        if char in ('“', '‘'):
            quote_stack.append(char)
        elif quote_stack:
            if char == '”' and quote_stack[-1] == '“':
                quote_stack.pop()
            elif char == '’' and quote_stack[-1] == '‘':
                quote_stack.pop()
        
        # 判断句子终结符
        if char in ('。', '！', '？', '……'):
                
            # 检查后引号终结符
            if i+1 < n and text[i+1] in ('”', '’'):
                if quote_stack and ((text[i+1] == '”' and quote_stack[-1] == '“') or 
                                  (text[i+1] == '’' and quote_stack[-1] == '‘')):
                    buffer.append(text[i+1])
                    quote_stack.pop()
                    i += 1
            
            # 判断是否终结句子
            # if not quote_stack or has_closing_quote:
            sentences.append(''.join(buffer))
            buffer = []
        
        i += 1
    
    if ''.join(buffer):  # 处理最后未完结内容
        print(buffer)
        sentences.append(''.join(buffer))
    
    return sentences

# 测试示例
text = """
他对小张说：“去年冬天的时候，我想离开长沙。老林劝阻我说：‘能不能玩几个月再走？陈亮昨天对我说：“前线形势不好，这几个月恐怕有大变化。最好等明年再参军。”当时我就想和你说，再等几个月。没想到你那么心急。’我没想到老林会出言挽留我。以他的性子，这是很少见的。”
"""

for i, sent in enumerate(split_chinese_sentences(text.strip("\n")), 1):
    print(f"句子{i}: {sent.strip()}")

句子1: 他对小张说：“去年冬天的时候，我想离开长沙。
句子2: 老林劝阻我说：‘能不能玩几个月再走？
句子3: 陈亮昨天对我说：“前线形势不好，这几个月恐怕有大变化。
句子4: 最好等明年再参军。”
句子5: 当时我就想和你说，再等几个月。
句子6: 没想到你那么心急。’
句子7: 我没想到老林会出言挽留我。
句子8: 以他的性子，这是很少见的。”


In [47]:
kw = """余忆童稚时，能张目对日，明察秋毫，见藐小之物必细察其纹理，故时有物外之趣。夏蚊成雷，私拟作群鹤舞于空中，心之所向，则或千或百，果然鹤也；昂首观之，项为之强。又留蚊于素帐中，徐喷以烟，使之冲烟而飞鸣，作青云白鹤观，果如鹤唳云端，为之怡然称快。余常于土墙凹凸处，花台小草丛杂处，蹲其身，使与台齐；定神细视，以丛草为林，以虫蚊为兽，以土砾凸者为丘，凹者为壑，神游其中，怡然自得。一日，见二虫斗草间，观之，兴正浓，忽有庞然大物，拔山倒树而来，盖一癞虾蟆，舌一吐而二虫尽为所吞。余年幼，方出神，不觉呀然一惊。神定，捉虾蟆，鞭数十，驱之别院。"""
kw = """混沌初开，乾坤始奠。

气之清轻者上浮为天，气之浊重者下凝为地。

日月五星，谓之七曜；天地与人，谓之三才。

北斗为帝车之象，北辰是太一所居。

叁星为白虎之体，大火是苍龙之心。

二十八宿，是日月之旅舍。黄道白道，载日月之行迹。

叁商二星，其出没不相见。牛女两宿，惟七夕一相逢。

后羿射日，女娲补天。羲和置闰，授民以时。

黄帝画野，始分都邑。夏禹治水，初奠山川。

宇宙之江山不改，古今之称谓各殊。

北京原属幽燕，金台是其异号；南京原为建业，金陵为其别名。

浙江是武林之区，原为越国；江西是豫章之地，昔从九江。

福建省属闽中，湖广地名三楚。

东鲁西鲁，即山东山西之分；东粤西粤，乃广东广西之域。

河南在华夏之中，故曰中州；陕西即长安之地，原为秦境。

四川为西蜀，云南为古滇。

贵州省近蛮方，自古名为黔地。

东岳泰山，西岳华山，南岳衡山，北岳恒山，中岳嵩山，此为天下之五岳。

饶州之鄱阳，岳州之青草，润州之丹阳，鄂州之洞庭，苏州之太湖，此为天下之五湖。

沧海桑田，谓世事之多变。河清海晏，兆天下之升平。

道不拾遗，由在上有善政。途通天堑，知中国有圣人。"""
kw = """
子曰：“学而时习之，不亦说乎？有朋自远方来，不亦乐乎？人不知而不愠，不亦君子乎？”

曾子曰：“吾日三省吾身：为人谋而不忠乎？与朋友交而不信乎？传不习乎？”

子曰：“吾十有五而志于学，三十而立，四十而不惑，五十而知天命，六十而耳顺，七十而从心所欲，不逾矩。”

子曰：“温故而知新，可以为师矣。”

子曰：“学而不思则罔，思而不学则殆。”

子曰：“贤哉，回也！一箪食，一瓢饮，在陋巷，人不堪其忧，回也不改其乐。贤哉，回也！”

子曰：“知之者不如好之者，好之者不如乐之者。”

子曰：“饭疏食，饮水，曲肱而枕之，乐亦在其中矣。不义而富且贵，于我如浮云。”

子曰：“三人行，必有我师焉。择其善者而从之，其不善者而改之。”

子在川上曰：“逝者如斯夫，不舍昼夜。”

子曰：“三军可夺帅也，匹夫不可夺志也。”

子夏曰：“博学而笃志，切问而近思，仁在其中矣。”"""
kw = """谢太傅寒雪日内集，与儿女讲论文义。俄而雪骤，公欣然曰：“白雪纷纷何

所似？”兄子胡儿曰：“撒盐空中差可拟。”兄女曰：“未若柳絮因风起。”公大笑

乐。即公大兄无奕女，左将军王凝之妻也。"""
kw = """陈太丘和朋友相约同行，约定的时间在中午。过了中午朋友还没有到，陈太

丘不再等候他而离开了，陈太丘离开后朋友才到。元方当时年龄七岁，在门外玩

耍。陈太丘的朋友问元方：“您的父亲在吗？”元方回答道：“我父亲等了您很久，

您却还没有到，已经离开了。”友人便生气地说道：“真不是人啊！和别人相约同

行，却丢下我先走了。”元方说：“您与我父亲约在正午，正午您没到，就是不讲

信用；对着孩子骂父亲，就是没有礼貌。”朋友感到惭愧，下了车想去拉元方的

手，元方头也不回地走进家门。"""
kw = "".join(kw.split("\n"))
print(kw)
pur = []
nsent = 0
for zi in kw:
    if zi not in 中文标点:
        pur.append(zi)
    if zi in "。！？":
        nsent += 1

len(pur), nsent

陈太丘和朋友相约同行，约定的时间在中午。过了中午朋友还没有到，陈太丘不再等候他而离开了，陈太丘离开后朋友才到。元方当时年龄七岁，在门外玩耍。陈太丘的朋友问元方：“您的父亲在吗？”元方回答道：“我父亲等了您很久，您却还没有到，已经离开了。”友人便生气地说道：“真不是人啊！和别人相约同行，却丢下我先走了。”元方说：“您与我父亲约在正午，正午您没到，就是不讲信用；对着孩子骂父亲，就是没有礼貌。”朋友感到惭愧，下了车想去拉元方的手，元方头也不回地走进家门。


(192, 9)

In [43]:
虚词 = ['而', '何', '乎', '乃', '其', '且', '若', '所', '为', '焉', '也', '以', '因', '于', '与', '则', '者', '之', '安', '但', '耳', '夫', '盖', '故', '或', '即', '既', '然', '虽', '遂', '惟', '已', '矣', '犹', '哉']
nice_print(虚词,7)

['而', '何', '乎', '乃', '其', '且', '若']
['所', '为', '焉', '也', '以', '因', '于']
['与', '则', '者', '之', '安', '但', '耳']
['夫', '盖', '故', '或', '即', '既', '然']
['虽', '遂', '惟', '已', '矣', '犹', '哉']


In [41]:
with open("../src/小学/文言实词.md", encoding="utf-8") as f:
    lines = f.read().split("\n\n")

shici = {}
numero = tuple("0123456789")
curr = None
for line in lines:
    if not line.startswith(numero) and len(line) < 2:
        zi = line[0]
        if zi in shici:
            print(f'{zi}字已经存在。')
            break
        else:
            shici[zi] = {}
        curr = zi
    elif curr and line.startswith(numero) and "（" in line:
        elements = line.split(".")
        if len(elements) > 1:
            elements = elements[1].split("（")
            yi = elements[1].rstrip("）").rstrip()
            liz = elements[0]
            shici[curr][yi] = liz
        else:
            print(line)
            print(elements)
            break  
    
print(list(shici.keys()))
print(len(shici))
dump_cn_json_compact("../src/中学/文言实词120.json", shici)



['安', '爱', '倍', '被', '本', '鄙', '兵', '病', '察', '朝', '曾', '乘', '辞', '从', '诚', '除', '殆', '当', '道', '诸', '贼', '族', '卒', '走', '左', '坐', '归', '得', '度', '非', '复', '负', '盖', '知', '致', '质', '治', '王', '望', '恶', '微', '悉', '相', '谢', '信', '兴', '行', '幸', '修', '徐', '许', '阳', '要', '宜', '遗', '贻', '易', '阴', '右', '再', '造', '故', '固', '顾', '国', '过', '何', '恨', '胡', '患', '或', '汤', '涕', '徒', '亡', '适', '书', '孰', '属', '数', '率', '说', '私', '素', '疾', '及', '即', '既', '假', '间', '见', '解', '就', '举', '绝', '堪', '克', '类', '胜', '识', '使', '是', '穷', '去', '劝', '却', '如', '若', '善', '少', '涉', '怜', '弥', '莫', '乃', '内', '迁', '请', '期', '奇']
120


In [31]:
实词 = ['爱', '安', '被', '倍', '本', '鄙', '兵', '病', '察', '朝', '曾', '乘', '诚', '除', '辞', '从', '殆', '当', '道', '得', '度', '非', '复', '负', '盖', '故', '顾', '固', '归', '国', '过', '何', '恨', '胡', '患', '或', '疾', '及', '即', '既', '假', '间', '见', '解', '就', '举', '绝', '堪', '克', '类', '怜', '弥', '莫', '乃', '内', '期', '奇', '迁', '请', '穷', '去', '劝', '却', '如', '若', '善', '少', '涉', '胜', '识', '使', '是', '适', '书', '孰', '属', '数', '率', '说', '私', '素', '汤', '涕', '徒', '亡', '王', '望', '恶', '微', '悉', '相', '谢', '信', '兴', '行', '幸', '修', '徐', '许', '阳', '要', '宜', '遗', '贻', '易', '阴', '右', '再', '造', '知', '致', '质', '治', '诸', '贼', '族', '卒', '走', '左', '坐']

for ci in 实词:
    if ci not in shici:
        print(ci)

朝
乘


In [69]:
s = "副词。"
s.rstrip(中文标点)

'副词'

In [87]:
中文数字 = "一二三四五六七八九十"
圆圈数字 = "①②③④⑤⑥⑦⑧⑨"



with open("../src/小学/文言虚词.md", encoding="utf-8") as f:
    lines = f.read().split("\n\n")

xuci = {}
i = 0
for line in lines:
    if len(line) > 2:
        i += 1
        match = re.match(r'^[' + 中文数字 + r']+、(.*)', line)
        if match:
            curr = match.group(1)
            i = 0
            xuci[curr] = []
        elif line.startswith("（") and line[1] in 中文数字:
            i = 0
            xuci[curr].append({"genre": line.split("）")[1].rstrip(中文标点), "yis": []})
        else:
            match1 = re.match(r'^\d+\.(.*)', line)
            match2 = re.match(r'^[' + 圆圈数字 + r'](.*)', line)
            if match1 and len(xuci[curr]):
                xuci[curr][-1]["yis"].append({"sense": match1.group(1).strip(), "ex": []})
            if match2 and len(xuci[curr][-1]["yis"]):
                xuci[curr][-1]["yis"][-1]["ex"].append(match2.group(1).strip())
            if not match1 and i == 1 and len(xuci[curr]):
                xuci[curr][-1]["remark"] = line

        
dump_cn_json_compact("../src/中学/文言虚词.json", xuci)
# len(xuci)

In [88]:
# 检查是否存在没有例子的义项
for zi in xuci:
    for content in xuci[zi]:
        for yi in content["yis"]:
            if not len(yi["ex"]):
                print(zi, content["genre"], yi["sense"])

# 统计各个字的义项数以及例子数
stats = {}
for zi in xuci:
    stats[zi] = {"genres": [], "yis": [], "ex": []}
    for content in xuci[zi]:
        stats[zi]["genres"].append(content["genre"])
        stats[zi]["yis"].append(len(content["yis"]))
        stats[zi]["ex"].append([])
        for yi in content["yis"]:
            stats[zi]["ex"][-1].append(len(yi["ex"]))

stats

{'而': {'genres': ['连词', '代词'],
  'yis': [8, 2],
  'ex': [[5, 4, 3, 4, 4, 3, 5, 3], [5, 1]]},
 '何': {'genres': ['疑问代词', '副词'], 'yis': [3, 2], 'ex': [[3, 3, 3], [3, 4]]},
 '乎': {'genres': ['句末语气助词', '结构助词', '介词'],
  'yis': [4, 1, 1],
  'ex': [[3, 3, 4, 2], [6], [7]]},
 '乃': {'genres': ['副词', '代词'], 'yis': [4, 2], 'ex': [[5, 3, 1, 4], [2, 2]]},
 '其': {'genres': ['代词', '副词', '连词', '助词'],
  'yis': [5, 3, 2, 1],
  'ex': [[4, 2, 3, 3, 3], [5, 4, 4], [3, 2], [2]]},
 '且': {'genres': ['连词', '副词', '助词'],
  'yis': [3, 2, 1],
  'ex': [[3, 2, 5], [6, 2], [1]]},
 '若': {'genres': ['动词', '代词', '连词', '助词'],
  'yis': [2, 2, 1, 1],
  'ex': [[3, 3], [4, 2], [2], [2]]},
 '所': {'genres': ['名词', '助词'], 'yis': [1, 3], 'ex': [[3], [4, 2, 3]]},
 '为': {'genres': ['动词', '介词', '助词'],
  'yis': [3, 4, 1],
  'ex': [[9, 2, 2], [4, 4, 3, 1], [3]]},
 '焉': {'genres': ['兼词', '代词', '助词'],
  'yis': [2, 2, 2],
  'ex': [[9, 2], [4, 5], [4, 5]]},
 '也': {'genres': ['语气助词'], 'yis': [3], 'ex': [[4, 6, 4]]},
 '以': {'genres': ['介词',

In [9]:
# 替换为注释格式
refs = """楚之同姓：楚王族本姓芈，楚武王熊通的儿子瑕封于屈，他的后代遂以屈为姓，瑕是屈原的祖先。楚国王族的同姓。屈、景、昭氏都是楚国的王族同姓。
楚怀王：楚威王的儿子，名熊槐（公元前328年至前299年在位）。
左徒：楚国官名，职位仅次于令尹。
上官大夫：楚大夫。
《离骚》：屈原的代表作，自叙生平的长篇抒情诗。关于诗题，后人有二说。一释“离”为“罹”的通假字，离骚就是遭受忧患。二是释“离”为离别，离骚就是离别的忧愁。
帝喾：喾，古代传说中的帝王。相传是黄帝的曾孙，号高辛氏。
齐桓：即齐桓公（公元前685年至前643年在位），名小白，春秋五霸之一。
汤：商朝的开国君主。
武：指周武王，灭商建立西周王朝。
从：同“纵”。从亲，合纵相亲。当时楚、齐等六国联合抗秦，称为合纵，楚怀王曾为纵长。
惠王：秦惠王（公元前337年至311年在位）。
张仪：魏人，主张“连横”，游说六国事奉秦国，为秦惠王所重。
商、於：秦地名。商，在今陕西商州市东南。於，在今河南内乡东。
丹、淅：二水名。丹水发源于陕西商州市西北，东南流入河南。淅水，发源于南卢氏县，南流而入丹水。屈匄：（gài）：楚大将军。汉中：今湖北西北部、陕西东南部一带。
蓝田：秦县名，在今陕西蓝田西。
邓：春秋时蔡地，后属楚，在今河南邓州市一带。
明年：指楚怀王十八年（公元前311年）。
靳尚：楚大夫。一说即上文的上官大夫。
唐昧：楚将。楚怀王二十八年（公元前301年），秦、齐、韩、魏攻楚，杀唐昧。
秦昭王：秦惠王之子，公元前306年至前251年在位。
武关：秦国的南关，在今陕西省商州市东。
顷襄王：名熊横，公元前298年至前262年在位。令尹：楚国的最高行政长官。
虽放流：以下关于屈原流放的记叙，时间上有矛盾，文意也不连贯，可能有脱误。
世：三十年为一世。
《易》：即《周易》，又称《易经》。这里引用的是《易经·井卦》的爻辞。
三闾大夫：楚国掌管王族屈、景、昭三氏事务的官。
《怀沙》：在今本《楚辞》中，是《九章》的一篇。
汨罗：江名，在湖南东北部，流经汨罗县入洞庭湖。
宋玉：相传为楚顷襄王时人，屈原的弟子，有《九辩》等作品传世。唐勒、景差：约与宋玉同时，都是当时的词赋家。
“数十年”句：公元前223年秦灭楚。
贾生：即贾谊（公元前200年前168年），洛阳（今河南洛阳东）人。西汉政论家、文学家。
长沙王：指吴差，汉朝开国功臣吴芮的玄孙。太傅：君王的辅助官员。
湘水：在今湖南省境内，流入洞庭湖。书：指贾谊所写的《吊屈原赋》。
太史公：司马迁自称。
《天问》、《招魂》、《哀郢》：都是屈原的作品。一说《招魂》为宋玉所作。《哀郢》是《九章》中的一篇。
《鵩鸟赋》：贾谊所作。"""

res = []
i = 0
for line in refs.split("\n"):
    head, tail = headtail(line, "：")
    i += 1
    res.append(f"N{i}：〔{head}〕{tail}")
    
print("\n\n".join(res))

N1：〔楚之同姓〕楚王族本姓芈，楚武王熊通的儿子瑕封于屈，他的后代遂以屈为姓，瑕是屈原的祖先。楚国王族的同姓。屈、景、昭氏都是楚国的王族同姓。

N2：〔楚怀王〕楚威王的儿子，名熊槐（公元前328年至前299年在位）。

N3：〔左徒〕楚国官名，职位仅次于令尹。

N4：〔上官大夫〕楚大夫。

N5：〔《离骚》〕屈原的代表作，自叙生平的长篇抒情诗。关于诗题，后人有二说。一释“离”为“罹”的通假字，离骚就是遭受忧患。二是释“离”为离别，离骚就是离别的忧愁。

N6：〔帝喾〕喾，古代传说中的帝王。相传是黄帝的曾孙，号高辛氏。

N7：〔齐桓〕即齐桓公（公元前685年至前643年在位），名小白，春秋五霸之一。

N8：〔汤〕商朝的开国君主。

N9：〔武〕指周武王，灭商建立西周王朝。

N10：〔从〕同“纵”。从亲，合纵相亲。当时楚、齐等六国联合抗秦，称为合纵，楚怀王曾为纵长。

N11：〔惠王〕秦惠王（公元前337年至311年在位）。

N12：〔张仪〕魏人，主张“连横”，游说六国事奉秦国，为秦惠王所重。

N13：〔商、於〕秦地名。商，在今陕西商州市东南。於，在今河南内乡东。

N14：〔丹、淅〕二水名。丹水发源于陕西商州市西北，东南流入河南。淅水，发源于南卢氏县，南流而入丹水。屈匄：（gài）：楚大将军。汉中：今湖北西北部、陕西东南部一带。

N15：〔蓝田〕秦县名，在今陕西蓝田西。

N16：〔邓〕春秋时蔡地，后属楚，在今河南邓州市一带。

N17：〔明年〕指楚怀王十八年（公元前311年）。

N18：〔靳尚〕楚大夫。一说即上文的上官大夫。

N19：〔唐昧〕楚将。楚怀王二十八年（公元前301年），秦、齐、韩、魏攻楚，杀唐昧。

N20：〔秦昭王〕秦惠王之子，公元前306年至前251年在位。

N21：〔武关〕秦国的南关，在今陕西省商州市东。

N22：〔顷襄王〕名熊横，公元前298年至前262年在位。令尹：楚国的最高行政长官。

N23：〔虽放流〕以下关于屈原流放的记叙，时间上有矛盾，文意也不连贯，可能有脱误。

N24：〔世〕三十年为一世。

N25：〔《易》〕即《周易》，又称《易经》。这里引用的是《易经·井卦》的爻辞。

N26：〔三闾大夫〕楚国掌管王族屈、景、昭三氏事务的官。

N27：〔《怀沙》〕在今本《楚辞》中，是《九章》的一篇。

N28：〔汨罗〕江

In [26]:
# 比较句子数

raw = """屈原者，名平，楚之同姓<footnote:N1>也。为楚怀王<footnote:N2>左徒<footnote:N3>。博闻强志，明于治乱，娴于辞令。入则与王图议国事，以出号令；出则接遇宾客，应对诸侯。王甚任之。

上官大夫与之同列，争宠而心害其能。怀王使屈原造为宪令，屈平属草稿未定。上官大夫见而欲夺之，屈平不与，因谗之曰：“王使屈平为令，众莫不知。每一令出，平伐其功，曰以为‘非我莫能为也。’”王怒而疏屈平。

屈平疾王听之不聪也，谗谄之蔽明也，邪曲之害公也，方正之不容也，故忧愁幽思而作《离骚》。“离骚”者，犹离忧也。夫天者，人之始也；父母者，人之本也。人穷则反本，故劳苦倦极，未尝不呼天也；疾痛惨怛，未尝不呼父母也。屈平正道直行，竭忠尽智以事其君，谗人间之，可谓穷矣。信而见疑，忠而被谤，能无怨乎？屈平之作《离骚》，盖自怨生也。《国风》好色而不淫，《小雅》怨诽而不乱。若《离骚》者，可谓兼之矣。上称帝喾，下道齐桓，中述汤、武，以刺世事。明道德之广崇，治乱之条贯，靡不毕见。其文约，其辞微，其志洁，其行廉。其称文小而其指极大，举类迩而见义远。其志洁，故其称物芳；其行廉，故死而不容。自疏濯淖污泥之中，蝉蜕于浊秽，以浮游尘埃之外，不获世之滋垢，皭然泥而不滓者也。推此志也，虽与日月争光可也。

屈平既绌，其后秦欲伐齐。齐与楚从亲，惠王患之。乃令张仪详去秦，厚币委质事楚，曰：“秦甚憎齐，齐与楚从亲，楚诚能绝齐，秦愿献商、於之地六百里。”楚怀王贪而信张仪，遂绝齐，使使如秦受地。张仪诈之曰：“仪与王约六里，不闻六百里。”楚使怒去，归告怀王。怀王怒，大兴师伐秦。秦发兵击之，大破楚师于丹、淅，斩首八万，虏楚将屈匄，遂取楚之汉中地。怀王乃悉发国中兵，以深入击秦，战于蓝田。魏闻之，袭楚至邓。楚兵惧，自秦归。而齐竟怒，不救楚，楚大困。

明年，秦割汉中地与楚以和。楚王曰：“不愿得地，愿得张仪而甘心焉。”张仪闻，乃曰：“以一仪而当汉中地，臣请往如楚。”如楚，又因厚币用事者臣靳尚，而设诡辩于怀王之宠姬郑袖。怀王竟听郑袖，复释去张仪。是时屈原既疏，不复在位，使于齐，顾反，谏怀王曰：“何不杀张仪？”怀王悔，追张仪，不及。

其后，诸侯共击楚，大破之，杀其将唐昧。

时秦昭王与楚婚，欲与怀王会。怀王欲行，屈平曰：“秦，虎狼之国，不可信，不如毋行。”怀王稚子子兰劝王行：“奈何绝秦欢！”怀王卒行。入武关，秦伏兵绝其后，因留怀王，以求割地。怀王怒，不听。亡走赵，赵不内。复之秦，竟死于秦而归葬。

长子顷襄王立，以其弟子兰为令尹。楚人既咎子兰以劝怀王入秦而不反也。屈平既嫉之，虽放流，眷顾楚国，系心怀王，不忘欲反。冀幸君之一悟，俗之一改也。其存君兴国，而欲反覆之，一篇之中，三致志焉。然终无可奈何，故不可以反。卒以此见怀王之终不悟也。

人君无愚、智、贤、不肖，莫不欲求忠以自为，举贤以自佐。然亡国破家相随属，而圣君治国累世而不见者，其所谓忠者不忠，而所谓贤者不贤也。怀王以不知忠臣之分，故内惑于郑袖，外欺于张仪，疏屈平而信上官大夫、令尹子兰，兵挫地削，亡其六郡，身客死于秦，为天下笑。此不知人之祸也。易曰：“井泄不食，为我心恻，可以汲。王明，并受其福。”王之不明，岂足福哉！

令尹子兰闻之，大怒，卒使上官大夫短屈原于顷襄王。顷襄王怒而迁之。

屈原至于江滨，被发行吟泽畔，颜色憔悴，形容枯槁。渔父见而问之曰：“子非三闾大夫欤？何故而至此？”屈原曰：“举世混浊而我独清，众人皆醉而我独醒，是以见放。”渔父曰：“夫圣人者，不凝滞于物，而能与世推移。举世混浊，何不随其流而扬其波？众人皆醉，何不哺其糟而啜其醨？何故怀瑾握瑜，而自令见放为？”屈原曰：“吾闻之，新沐者必弹冠，新浴者必振衣。人又谁能以身之察察，受物之汶汶者乎？宁赴常流而葬乎江鱼腹中耳。又安能以皓皓之白，而蒙世之温蠖乎？”乃作《怀沙》之赋。于是怀石，遂自投汨罗以死。

屈原既死之后，楚有宋玉、唐勒、景差之徒者，皆好辞而以赋见称。然皆祖屈原之从容辞令，终莫敢直谏。其后楚日以削，数十年竟为秦所灭。

太史公曰：“余读《离骚》《天问》《招魂》《哀郢》，悲其志。适长沙，观屈原所自沉渊，未尝不垂涕，想见其为人。及见贾生吊之，又怪屈原以彼其材游诸侯，何国不容，而自令若是！读《鵩鸟赋》，同死生，轻去就，又爽然自失矣。”"""

yiwen = """屈原，名字叫平，与楚国的王族同姓，做楚怀王的左徒。（他）见识广博，记忆力很强，通晓国家治乱的道理，擅长外交辞令。对内，同楚王谋划商讨国家大事，颁发号令；对外，接待宾客，应酬答对各国诸侯。楚王很信任他。

上官大夫和他职位相等，想争得楚王对他的宠爱，便心里嫉妒屈原的贤能。楚怀王派屈原制定国家的法令，屈原编写的草稿尚未定稿。上官大夫看见了，就想把草稿强取为己有。屈原不赞同。上官大夫就谗毁他说：“君王让屈原制定法令，大家没人不知道的，每出一道法令，屈原就炫耀自己的功劳，说：‘除了我，没有人能制定法令了’。”楚王听了很生气，因而疏远了屈原。

屈原痛心楚怀王听信谗言，不能分辨是非，谄媚国君的人遮蔽了楚怀王的明见，品行不正的小人损害国家，端方正直的人不被昏君谗臣所容，所以忧愁深思，就创作了《离骚》。“离骚”，就是遭遇忧愁的意思。上天，是人的原始；父母，是人的根本。人处境困难时，总是要追念上天和父母（希望给以援助），所以劳累疲倦时，没有不呼叫上天的；病痛和内心悲伤时，没有不呼叫父母的。屈原使（自己）道德端正，使（自己）品行正直，竭尽忠心用尽智慧来侍奉他的国君，却被小人离间，可以说处境很困难。诚信而被怀疑，尽忠却被诽谤，能没有怨愤吗？屈原作《离骚》，大概是自己的怨愤所引起的。《诗经》中的《国风》，写男女恋情而不过度，《小雅》有怨刺之言，但不直接愤怒。屈原的《离骚》诗，则两者之美兼而有之。（他）远古提到帝喾，近古提到齐桓公，中古提到商汤、周武王，利用古代帝王这些事用来讽刺当世社会。阐明道德的广大崇高，治乱的条理，没有不全表现出来的。他的文章简约，语言含蓄，他的志趣高洁，行为正直。就其文字来看，不过是寻常事情，但是它的旨趣是极大的，列举的是近事，而表达的意思却十分深远。他的志趣高洁，所以作品中多用美人芳草作比喻；他的行为正直，所以至死不容于世。他自动地远离污泥浊水，像蝉脱壳那样摆脱污秽环境，以便超脱世俗之外，不沾染尘世的污垢，出于污泥而不染，依旧保持高洁的品德，推赞这种志行，即使同日月争光都可以。

屈原被罢免了，后来秦国准备攻打齐国。齐国和楚国结成合纵联盟互相亲善，秦惠王对此担忧。就派张仪假装脱离秦国，用厚礼和信物呈献给楚王，对怀王说：“秦国非常憎恨齐国，齐国与楚国却合纵相亲，如果楚国确实能和齐国绝交，秦国愿意献上商、於之间的六百里土地。”楚怀王起了贪心，信任了张仪，就和齐国绝交，然后派使者到秦国接受土地。张仪抵赖说：“我和楚王约定的只是六里，没有听说过六百里。”楚国使者愤怒地离开秦国，回去报告怀王。怀王发怒，大规模出动军队去讨伐秦国。秦国发兵反击，在丹水和淅水一带大破楚军，杀了八万人，俘虏了楚国的大将屈匄，于是夺取了楚国的汉中一带。怀王又发动全国的兵力，深入秦地攻打秦国，交战于蓝田。魏国听到这一情况，袭击楚国一直打到邓地。楚军恐惧，从秦国撤退。齐国终于因为怀恨楚国，不来援救，楚国处境极端困窘。

第二年，秦国割汉中之地与楚国讲和。楚王说：“我不愿得到土地，只希望得到张仪就甘心了。”张仪听说后，就说：“用一个张仪来抵当汉中地方，我请求到楚国去。”到了楚国，他又用丰厚的礼品贿赂当权的大臣靳尚，通过他在怀王宠姬郑袖面前编造了一套谎话。怀王竟然听信郑袖，又放走了张仪。这时屈原已被疏远，不在朝中任职，出使在齐国，回来后，劝谏怀王说：“为什么不杀张仪？”怀王很后悔，派人追张仪，已经来不及了。

后来，各国诸侯联合攻打楚国，大败楚军，杀了楚国将领唐昧。

当时秦昭王与楚国通婚，要求和怀王会面。怀王想去，屈原说：“秦国是虎狼一样的国家，不可信任，不如不去。”怀王的小儿子子兰劝怀王去，说：“怎么可以断绝和秦国的友好关系！”怀王终于前往。一进入武关，秦国的伏兵就截断了他的后路，于是扣留怀王，强求割让土地。怀王很愤怒，不听秦国的要挟。他逃往赵国，赵国不肯接纳。只好又到秦国，最后死在秦国，尸体运回楚国安葬。

怀王的长子顷襄王即位，任用他的弟弟子兰为令尹。楚国人都抱怨子兰，因为他劝怀王入秦而最终未能回来。屈原也为此怨恨子兰，虽然流放在外，仍然眷恋着楚国，心里挂念着怀王，念念不忘返回朝廷。他希望国君总有一天醒悟，世俗总有一天改变。屈原关怀君王，想振兴国家改变楚国的形势，一篇作品中，都再三表现出来这种想法。然而终于无可奈何，所以不能够返回朝廷。由此可以看出怀王始终没有觉悟啊。

国君无论愚笨或明智、贤明或昏庸，没有不想求得忠臣来为自己服务，选拔贤才来辅助自己的。然而国破家亡的事接连发生，而圣明君主治理好国家的多少世代也没有出现，这是因为所谓忠臣并不忠，所谓贤臣并不贤。怀王因为不明白忠臣的职分，所以在内被郑袖所迷惑，在外被张仪所欺骗，疏远屈原而信任上官大夫和令尹子兰，军队被挫败，土地被削减，失去了六个郡，自己也被扣留死在秦国，为天下人所耻笑。这是不了解人（而导致）的祸害。《易经》说：“水井淘干净了，但是没人饮用，（这）让我心里感到难过，（因为这是）可以汲取饮用的。君主如果贤明，大家都能得到幸福。”现在君主是这样的不贤明，哪里还谈得上幸福呢！

令尹子兰得知屈原怨恨他，非常愤怒，终于让上官大夫在顷襄王面前说屈原的坏话。顷襄王发怒，就放逐了屈原。

屈原到了江滨，披散头发，在水泽边一面走，一面吟咏着，脸色憔悴，形体面貌像枯死的树木一样毫无生气。渔父看见他，便问道：“您不是三闾大夫吗？为什么来到这儿？”屈原说：“整个世界都是混浊的，只有我一人清白；众人都沉醉，只有我一人清醒，因此被放逐。”渔父说：“聪明贤哲的人，不受外界事物的束缚，而能够随着世俗变化。整个世界都混浊，为什么不随大流而且推波助澜呢？众人都沉醉，为什么不吃点酒糟，喝点薄酒？为什么要怀抱美玉一般的品质，却使自己被放逐呢？”屈原说：“我听说，刚洗过头的一定要弹去帽上的灰沙，刚洗过澡的一定要抖掉衣上的尘土。谁能让自己清白的身躯，蒙受外物的污染呢？宁可投入长流的大江而葬身于江鱼的腹中。又哪能使自己高洁的品质，去蒙受世俗的尘垢呢？”于是他写了《怀沙》赋。因此抱着石头，就自投汨罗江而死。

屈原死了以后，楚国（还）有宋玉、唐勒、景差一些人，都爱好文学，由于擅长写赋受到人们称赞。然而（他们）都效法屈原的委婉文辞，始终没有人敢于直谏。从这以后，楚国一天比一天缩小，几十年后，终于被秦国所灭亡。

太史公说：我读《离骚》《天问》《招魂》《哀郢》，为他的志向不能实现而悲伤。到长沙，经过屈原自沉的地方，未尝不流下眼泪，追怀他的为人。看到贾谊凭吊他的文章，文中又责怪屈原如果凭他的才能去游说诸侯，哪个国家不会容纳，却自己选择了这样的道路！读了《鵩鸟赋》，把生和死等同看待，认为被贬和任用是不重要的，这又使我感到茫茫然失落什么了。"""

def print_paras(sent1, sent2):
    n1 = len(sent1)
    n2 = len(sent2)
    j = 0
    for i in range(max(n1, n2)):
        if i < n1:
            s1 = sent1[i]
        else:
            s1 = ""
        if i < n2:
            s2 = sent2[i]
        else:
            s2 = ""
        j += 1
        print(f"原文{j:2d}：{s1}")
        print(f"译文{j:2d}：{s2}")

text1 = raw.split("\n\n")
text2 = yiwen.split("\n\n")
if len(text1) != len(text2):
    print(f"段落数目不符：原文{len(text1)}段，译文{len(text2)}段。")
else:
    sents = []
    i = 0
    for para1, para2 in zip(text1, text2):
        sent1 = tuple(split_chinese_sentences(para1))
        sent2 = tuple(split_chinese_sentences(para2))
        sents.append((sent1, sent2))
        n1 = len(sent1)
        n2 = len(sent2)
        i += 1
        pm = "+" if n2 > n1 else ""
        pm += str(n2 - n1)
        if n1 == n2:
            pm = "=0="
        print(f"第{i}段\t原文{n1:2d}句，译文{n2:2d}句。\t{pm}")

print_paras(*sents[10])


第1段	原文 5句，译文 4句。	-1
第2段	原文 5句，译文 6句。	+1
第3段	原文16句，译文15句。	-1
第4段	原文12句，译文12句。	=0=
第5段	原文 7句，译文 7句。	=0=
第6段	原文 1句，译文 1句。	=0=
第7段	原文 8句，译文 8句。	=0=
第8段	原文 7句，译文 7句。	=0=
第9段	原文 7句，译文 7句。	=0=
第10段	原文 2句，译文 2句。	=0=
第11段	原文14句，译文14句。	=0=
第12段	原文 3句，译文 3句。	=0=
第13段	原文 4句，译文 4句。	=0=
原文 1：屈原至于江滨，被发行吟泽畔，颜色憔悴，形容枯槁。
译文 1：屈原到了江滨，披散头发，在水泽边一面走，一面吟咏着，脸色憔悴，形体面貌像枯死的树木一样毫无生气。
原文 2：渔父见而问之曰：“子非三闾大夫欤？
译文 2：渔父看见他，便问道：“您不是三闾大夫吗？
原文 3：何故而至此？”
译文 3：为什么来到这儿？”
原文 4：屈原曰：“举世混浊而我独清，众人皆醉而我独醒，是以见放。”
译文 4：屈原说：“整个世界都是混浊的，只有我一人清白；众人都沉醉，只有我一人清醒，因此被放逐。”
原文 5：渔父曰：“夫圣人者，不凝滞于物，而能与世推移。
译文 5：渔父说：“聪明贤哲的人，不受外界事物的束缚，而能够随着世俗变化。
原文 6：举世混浊，何不随其流而扬其波？
译文 6：整个世界都混浊，为什么不随大流而且推波助澜呢？
原文 7：众人皆醉，何不哺其糟而啜其醨？
译文 7：众人都沉醉，为什么不吃点酒糟，喝点薄酒？
原文 8：何故怀瑾握瑜，而自令见放为？”
译文 8：为什么要怀抱美玉一般的品质，却使自己被放逐呢？”
原文 9：屈原曰：“吾闻之，新沐者必弹冠，新浴者必振衣。
译文 9：屈原说：“我听说，刚洗过头的一定要弹去帽上的灰沙，刚洗过澡的一定要抖掉衣上的尘土。
原文10：人又谁能以身之察察，受物之汶汶者乎？
译文10：谁能让自己清白的身躯，蒙受外物的污染呢？
原文11：宁赴常流而葬乎江鱼腹中耳。
译文11：宁可投入长流的大江而葬身于江鱼的腹中。
原文12：又安能以皓皓之白，而蒙世之温蠖乎？”
译文12：又哪能使自己高洁的品质，去蒙受世俗的尘垢呢？”
原文13：乃作《怀沙》之赋。
译文13：于是他写了《怀沙》赋。
原文14：于是怀石，

## 文言文转tex

In [44]:
sorted([(2,1), (1,3), (1,1), (3,1),(2,2)])

[(1, 1), (1, 3), (2, 1), (2, 2), (3, 1)]

In [4]:
import json
import os
import matplotlib.pyplot as plt
import numpy as np
from outils import read, keys, load_cn_json, dump_cn_json, 中转数, 数转中, set_char_colors, nice_print, sort_dict_with, dump_cn_json_compact

def wrap(s, wrapper="{}", keep_wrapper=False):
    if s:
        return wrapper[0] + s + wrapper[-1]
    if keep_wrapper:
        return wrapper
    return ""

def make_params(params, wrapper="[]", sep=","):
    return wrap(sep.join(params), wrapper)

def wrap_env(name, content, params=[], param_wrapper="[]", param_sep=","):
    out = r"\begin" + wrap(name) + make_params(params, wrapper=param_wrapper, sep=param_sep) + "\n"
    lines = content[:-1].split("\n")  # presume content ends with \n
    for line in lines:
        out += "    " + line + "\n"
    out += r"\end" + wrap(name) + "\n"
    return out

def wrap_method(method, content="", wrapper="{}", keep_wrapper=True, params=[], param_wrapper="[]", param_sep=","):
    return '\\' + method + make_params(params, wrapper=param_wrapper, sep=param_sep) + wrap(content, wrapper=wrapper, keep_wrapper=keep_wrapper)

def zihao(n):
    return wrap_method("zihao", str(n))

def package_update_xcolor(packages, texts):
    xcolor = packages["xcolor"]
    xcolor["defined_colors"] = {}
    for _, text in texts.items():
        if "character_colors" in text:
            for key, val in text["character_colors"].items():
                xcolor["defined_colors"][key] = val
    packages["xcolor"] = xcolor

def make_ctex_env(document_class="ctexbook", document_class_params=("12pt", "UTF-8","openany"), packages={"ctex": [], "titlesec": []}, mainfont="Mona Sans Light", lineskip="4pt", parskip="10pt", title="标题", author="", date=False, toc=True):
    """make header and footer for ctexbook environment. 
    header
    1. documentclass and parameters 
    2. packages
    3. geometry and fonts
    4. package setups
    5. global typesettings
    6. begin document
    footer
    1. end document
    """
    # ## header ##

    # document class
    header = r"\documentclass"+ make_params(document_class_params) + wrap(document_class) + "\n"
    
    # packages
    packages_str = ""
    for name in packages:
        # print(package)
        package_declarations = ""
        if "declarations" in packages[name]:
            package_declarations = make_params(packages[name]['declarations'])
        packages_str += r"\usepackage" + package_declarations + wrap(name) + "\n"
    # print(packages_str)
    header += packages_str + "\n"

    # geometry <-- geometry package
    if "geometry" in packages:
        geometry = packages["geometry"]
        paper_type = geometry["paper_size"]
        paddings = geometry["paddings"]
        left = paddings["left"]
        right = paddings["right"]
        top = paddings["top"]
        bottom = paddings["bottom"]
        header += wrap_method("geometry", f"{paper_type}paper,left={left},right={right},top={top},bottom={bottom}") + "\n"
    
    # fonts
    header += r"\renewcommand{\footnotesize}{\fontsize{8.5pt}{10.5pt}\selectfont}" + "\n"
    header += wrap_method("setmainfont", mainfont) + "\n"
    header += r"\setCJKmainfont[BoldFont=STZhongsong]{汉字之美仿宋GBK 免费}" + "\n"
    header += r"\xeCJKDeclareCharClass{CJK}{`0 -> `9}" + "\n"  # apply CJK font to numbers
    header += r"\xeCJKsetup{AllowBreakBetweenPuncts=true}" + "\n"  # line alignment

    if "footmisc" in packages:
        footnote_settings_content = "".join([r"{\ding{"+str(192+i)+r"}}" for i in range(10)])
        footnote_settings = wrap_method("DefineFNsymbols", footnote_settings_content, params=["circled"], param_wrapper="{}")
        header += footnote_settings + "\n"
        header += wrap_method("setfnsymbol", "circled") + "\n"

    # package setups
    # xpinyin
    if "xpinyin" in packages:
        pyr = packages['xpinyin']['ratio']  # size ratio
        vsep = packages['xpinyin']['vsep']  # vertical gap
        vsep_str = "vsep={" + vsep + "}"
        hsep = packages['xpinyin']['hsep']  # horizontal gap
        hsep_str = "hsep={" + f"{hsep} plus {hsep}" + "}"
        header += wrap_method("xpinyinsetup", f"ratio={pyr},{hsep_str},{vsep_str}") + "\n"  # pinyin settings

    # hanzibox
    if "hanzibox" in packages:
        hanzibox = packages["hanzibox"]
        frametype = hanzibox['frametype']
        framelinewidth = hanzibox['framelinewidth']
        width = hanzibox['width'] if 'width' in hanzibox else "0cm"
        height = hanzibox['height'] if 'height' in hanzibox else "0cm"
        resize = hanzibox['resize'] if 'resize' in hanzibox else "none"
        framecolor = hanzibox["framecolor"]
        pinyinline = hanzibox['pinyinline']
        pinyinf = hanzibox['pinyinf']
        pinyincolor = hanzibox['pinyincolor']
        charcolor = hanzibox['charcolor']
        charf = "charf={" + hanzibox["charf"]["font"] + hanzibox["charf"]["fontsize"] + "}"
        header += wrap_method("hanziboxset", f"frametype={frametype},framelinewidth={framelinewidth},width={width},height={height},resize={resize},pinyinline={pinyinline},framecolor={framecolor},{charf},pinyinf={pinyinf},pinyincolor={pinyincolor},charcolor={charcolor}") + "\n"  # hanzibox settings

    # package setups
    # xcolor
    if "xcolor" in packages:
        defcolor_str = ""
        for key, (r, g, b) in packages["xcolor"]["defined_colors"].items():
            rgb_plate = f"{r},{g},{b}"
            defcolor_str += wrap_method("definecolor", key) + r"{RGB}{" + rgb_plate + r"}" + "\n"
        header += defcolor_str + "\n"

    # global typesettings
    # title format
    header += r"\titleformat{\chapter}{\zihao{-1}\bfseries}{ }{16pt}{}" + "\n"
    header += r"\titleformat{\section}{\zihao{-2}\bfseries}{ }{0pt}{}" + "\n"
    header += r"\title" + wrap(r"\zihao{0} \bfseries " + title) + "\n"
    # line and paragraph skips
    header += r"\setlength{\lineskip}{" + lineskip + "}\n"  # skip length after line
    header += r"\setlength{\parskip}{" + parskip + "}\n"  # extra skip for paragraphs 
    # front page format
    if author:  # author format
        header += r"\author{\zihao{2} \texttt" + wrap(author) + "}\n"
    else:
        header += r"\author{}" + "\n"
    if date:  # date format
        header += r"\date{\bfseries\today}" + "\n"
    else:
        header += r"\date{}" + "\n"
    
    # begin document
    header += r"\begin" + wrap("document") + "\n"
    header += r"\maketitle" + "\n"
    if toc:
        header += r"\tableofcontents" + "\n"
    header += r"\newpage" + "\n"
    
    # ## footer ##

    # end document
    footer = r"\end" + wrap("document") + "\n"
    return header, footer

def read_text(path, format="散文"):
    """Read raw text and formalize to json
    Inputs: 
    path (str): file path to the raw text.
    format (str): format of the text.
    Output:
    out (dict): a jsonifiable dictionary with formalized text.
    Example:
    out["format"]     : format of the text (in the sense of tex printing).
    out["genre"]      : genre and other tags of the text.
    out["content"]    : content of the text. A list of strings.
    out["grade"]      : recommanded student grade (for the purpose of eduation).
    out["title"]      : title of the text.
    out["author"]     : author of the text.
    out["remarks"]    : remarks concerning the text.
    out["footnotes"]  : footnotes of the content of the text.
    out["endnotes"]   : endnotes of the content of the text.
    out["vocabulary"] : vocabulary to learn (for the purpose of eduation).
    """
    lines = read(path)
    out = {}
    title = ""
    if len(lines) and len(lines[0]):
        author = ""
        grade = 0
        footnotes = []
        endnotes = []
        vocabulary = []
        remarks = []
        content = []
        out["format"] = format
        out["genre"] = [format]
        # return lines
        if format in ("散文", "书信", "小说", "剧本"):
            for line in lines:
                line0 = line.strip()
                if line0:
                    if not title:
                        title = line0
                    elif grade < 1 and line.startswith("年级："):
                        grade = int(line0[3:])
                    elif not author and line.startswith("作者："):
                        author = line0[3:]
                    elif line.startswith("备注："):
                        remarks.append(line0[3:])
                    elif line.startswith("注释："):
                        footnotes.append(line0[3:])
                    elif line.startswith("脚注："):
                        footnotes.append(line0[3:])
                    elif line.startswith("尾注："):
                        endnotes.append(line0[3:])
                    elif line.startswith("词汇："):
                        vocabulary.extend(line0[3:].split())
                    else:
                        content.append(line0)
        elif format == "诗歌":
            para = []
            for line in lines:
                line0 = line.strip()
                if line0:
                    if not title:
                        title = line0
                    elif grade < 1 and line.startswith("年级："):
                        grade = int(line0[3:])
                    elif not author and line.startswith("作者："):
                        author = line0[3:]
                    elif line.startswith("备注："):
                        remarks.append(line0[3:])
                    elif line.startswith("注释："):
                        footnotes.append(line0[3:])
                    elif line.startswith("脚注："):
                        footnotes.append(line0[3:])
                    elif line.startswith("尾注："):
                        endnotes.append(line0[3:])
                    elif line.startswith("词汇："):
                        vocabulary.extend(line0[3:].split())
                    else:
                        para.append(line0)
                elif len(para):
                    content.append("|#|".join(para))
                    para = []
            if len(para):
                content.append("|#|".join(para))
        # make footnotes dict
        footdict = {}
        i = 0
        keybase = "fn"
        content = "@".join(content)
        for note in footnotes:
            word = ""
            if note.startswith("〔"):
                word = note.split("〕")[0][1:]
                key = keybase + str(i+1)
                footdict[key] = note
                i += 1
            elif "〕" in note:  # key is already marked in the text with the format "\apost{a...}".
                key = note.split("〕")[0].split("〔")[0]
                footdict[key] = "".join(note.split(key)[1:])
            # print(word)
            if word:  # find the position to insert footnote and mark
                nfin = content.find(word) + len(word)
                content = content[:nfin] + r"\apost{" + key + "}" + content[nfin:]
        if "|#|" in content:
            content_new = []
            for para in content.split("@"):
                content_new.append(para.split("|#|"))
            content = content_new
        else:
            content = content.split("@")
        
        out["title"] = title
        out["author"] = author
        out["content"] = content
        out["remarks"] = remarks
        out["footnotes"] = footdict
        out["endnotes"] = endnotes
        out["vocabulary"] = vocabulary
        if grade:
            out["grade"] = grade
    return title, out

def text_content_to_tex_str(text, footnotes):
    """convert the content of a text to text string ready for tex.
    """ 
    out = text
    
    for key, value in footnotes.items():
        out = out.replace(f"<footnote:{key}>", "\\footnote{" + value["content"] + "}")

    return out

def endnotes_to_str(endnotes):
    """convert the endnotes to text string ready for tex."""
    out = ""
    notes = ""
    for pid, note in endnotes.items():
        notes += r"\item〔" + note["text"] + "〕" + note["note"] + "\n"
    if notes:
        out = r"\newpage" + "\n\n" + r"\textbf{注解}：" + "\n\n" + r"\vspace{-1em}" + "\n\n"
        out += wrap_env("itemize", r"\setlength\itemsep{-0.2em}" + "\n" + notes)
    return out

def shizi_to_str(zis, n=8):
    out = r"\clearpage" + "\n\n"
    boxes = ""
    i = 0
    for zi in zis:
        boxes += wrap_method("hanzibox", zi)
        i += 1
        if i == n:
            boxes += "\n\n"
            i = 0
    out += wrap_env("center", boxes + "\n\n")
    return out

def xiezi_to_str(zis, ncol=2, nex=4, hspace=1):
    out = ""
    boxes = ""
    i = 0
    for zi in zis:
        boxes += wrap_method("hanzibox", zi)
        for j in range(nex):
            boxes += wrap_method("hanzibox", "")
        i += 1
        if i == ncol:
            boxes += "\n\n"
            i = 0
        else:
            boxes += wrap_method("hspace", f"{hspace}em")
    out += boxes + "\n\n"
    # out += wrap_env("center", boxes + "\n\n")
    return out

def yiwen_to_str(sentences):
    keys = []
    for pid in sentences:
        p, s = pid.split("-")
        keys.append((int(p[1:]), int(s[1:])))

    keys = sorted(keys)
    out = ""
    for i, (p, s) in enumerate(keys):
        if i and (p > keys[i-1][0]): # next paragraph
            out += "\n\n"
        out += sentences[f"P{p}-S{s}"]["translation"]

    return out

def text_to_tex_str(text, typesettings={"font": {"title": {"size": 2}, "plaintext": {"size": "normalsize"}}, "vspaces": {"after_title": 12, "after_author": 6, "after_content": 6}}):
    """convert a text object to text string ready for tex
    """
    out = ""
    content = ""
    title = wrap_method("chapter", text["metadata"]["title"]) + "\n\n"
    content += title
    content += wrap_env(typesettings["font"]["plaintext"]["size"], "\n" + text_content_to_tex_str(text["content"]["raw"], text["annotations"]["footnotes"]) + "\n")
    content += "\n\n" + r"\newpage" + "\n\n" + r"\textbf{译文}：" + "\n\n" + r"\vspace{-1em}" + "\n\n"
    content += wrap_env(typesettings["font"]["plaintext"]["size"], "\n" + yiwen_to_str(text["annotations"]["sentences"]) + "\n\n")
    out += content + "\n\n"
    if len(text["teaching_materials"]["sentences"]):
        out += endnotes_to_str(text["teaching_materials"]["sentences"])
    return out

def add_text(texts, title, content, format="散文", tags=[]):
    """Add a text to the dictionary of texts.
    Inputs:
    texts (dict): dictionary of texts. title --> content.
    title (str): title of the text.
    content (dict): content of the text.
    format (str): format of the text.
    tags (list of str): tags to describe the text.
    Output:
    texts: updated dictionary of texts. 
    """
    if len(tags):
        content["genre"] = tags
    if format == "剧本":
        if title not in texts:
            script_keys = []
            for _, text in texts.items():
                if text["format"] == "剧本" and "key" in text:
                    script_keys.append(int(text["key"].split("-")[1]))
            if len(script_keys):
                script_key = "script-" + str(max(script_keys) + 1)
            else:
                script_key = "script-1"
                
        else:
            script_key = texts[title]["key"]
        name_set, color_set = set_char_colors(content["content"], script_key)
        
    texts[title] = content
    if format == "剧本":
        texts[title]["key"] = script_key
        texts[title]["characters"] = name_set
        texts[title]["character_colors"] = color_set
    return texts



In [5]:
# 打印页面设置：纸号，页边距等
geometry = {}
geometry["paper_size"] = "a5"  # 使用A5纸
paddings = {}  # 页边距
paddings["left"] = "1.4cm"
paddings["right"] = "1.4cm"
paddings["top"] = "2.3cm"
paddings["bottom"] = "2.3cm"
geometry["paddings"] = paddings

# 拼音设置： xpinyin宏包
pinyin = {}
pinyin["ratio"] = "0.5"
pinyin["hsep"] = ".6em"
pinyin["vsep"] = "1em"

# 田字格设置：hanzibox宏包
# \hanziboxset{frametype=咪,framelinewidth=0.5pt,width=1.0cm,resize=real,pinyinline=true,framecolor=red,charf={\kaishu\huge},pinyinf=\scriptsize,pinyincolor=green!30!black,charcolor=green!30!black}
hanzibox = {}
hanzibox["frametype"] = "咪"
hanzibox["framelinewidth"] = "0.5pt"
hanzibox["width"] = "1cm"
hanzibox["height"] = "1cm"
hanzibox["resize"] = "none"
hanzibox["pinyinline"] = "true"
hanzibox["framecolor"] = "red"
hanzibox["pinyinf"] = r"\scriptsize"
hanzibox["charf"] = {"font": r"\kaishu", "fontsize": r"\huge"}
hanzibox["pinyincolor"] = r"green!30!black"
hanzibox["charcolor"] = r"green!30!black"


In [6]:
packages = {}
packages["ctex"] = []
packages["titlesec"] = []
packages["xeCJK"] = []
packages["verse"] = []
packages["fontspec,xunicode,xltxtra"] = []
packages["xpinyin"] = pinyin
packages["geometry"] = geometry
packages["indentfirst"] = []
packages["pifont"] = []
packages["enumitem"] = []
packages["footmisc"] = {"declarations": ["perpage", "symbol*"]}
xcolor = {}
xcolor["declarations"] = ["table", "dvipsnames"]
packages["xcolor"] = xcolor

typesettings = {}
typesettings["vspaces"] = {"after_title": 36, "after_author": 16, "after_content": 16}
typesettings["font"] = {"plaintext": {"size": "normalsize"}}

In [7]:
booktitle = "中学古文课文集萃"
texts_cz = load_cn_json("../src/中学/中学古文阅读课文.json")

lineskip = "24pt"
parskip = "6pt"
package_update_xcolor(packages, texts_cz)
header, footer = make_ctex_env(packages=packages, title=booktitle, parskip=parskip, lineskip=lineskip)
with open("中学古文阅读课文集萃.tex", "w", encoding="utf-8") as f:
    f.write(header + "\n")
    for title, text in texts_cz.items():
        # print(title)
        if text["teaching_materials"]["grade"] > 0:
            f.write(text_to_tex_str(text, typesettings=typesettings) + "\n")
        # break
    f.write(footer)

In [56]:
title = "使琉球记"

texts_cz = load_cn_json("../src/中学/中学古文阅读课文.json")

lineskip = "24pt"
parskip = "6pt"
package_update_xcolor(packages, texts_cz)
header, footer = make_ctex_env(packages=packages, title=title, parskip=parskip, lineskip=lineskip)

with open("中学古文阅读课文.tex", "w", encoding="utf-8") as f:
    f.write(header + "\n")
    for tid, text in texts_cz.items():
        if text["metadata"]["title"] == title:
            f.write(text_to_tex_str(text, typesettings=typesettings) + "\n")
    f.write(footer)

In [65]:
from pdf2image import convert_from_path
import os

# Function to convert PDF to images
def pdf_to_images(pdf_path, output_folder, filename):
    # Convert PDF to list of PIL.Image
    images = convert_from_path(pdf_path)

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save each image in the list
    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f"{filename}_{i + 1}.png")  # Adjust extension as needed
        image.save(image_path, "PNG")  # Adjust format as needed
        print(f"Saved {image_path}")

# Example usage
title = r"使琉球记"
pdf_path = r"../语文/中学古文阅读课文.pdf"  # Replace with your PDF file path
output_folder = f"../语文/{title}/"  # Replace with desired output folder path
pdf_to_images(pdf_path, output_folder, title)

Saved ../语文/使琉球记/使琉球记_1.png
Saved ../语文/使琉球记/使琉球记_2.png
Saved ../语文/使琉球记/使琉球记_3.png
Saved ../语文/使琉球记/使琉球记_4.png
Saved ../语文/使琉球记/使琉球记_5.png
Saved ../语文/使琉球记/使琉球记_6.png
Saved ../语文/使琉球记/使琉球记_7.png
Saved ../语文/使琉球记/使琉球记_8.png
Saved ../语文/使琉球记/使琉球记_9.png
Saved ../语文/使琉球记/使琉球记_10.png
Saved ../语文/使琉球记/使琉球记_11.png
Saved ../语文/使琉球记/使琉球记_12.png
Saved ../语文/使琉球记/使琉球记_13.png
Saved ../语文/使琉球记/使琉球记_14.png
Saved ../语文/使琉球记/使琉球记_15.png
Saved ../语文/使琉球记/使琉球记_16.png
Saved ../语文/使琉球记/使琉球记_17.png
Saved ../语文/使琉球记/使琉球记_18.png
